In [1]:
%%bash
# Check if Terraform is installed
if ! command -v terraform &> /dev/null
then
    echo "Installing Terraform..."
    wget https://releases.hashicorp.com/terraform/1.13.0/terraform_1.13.0_linux_amd64.zip
    unzip terraform_1.13.0_linux_amd64.zip
    sudo mv terraform /usr/local/bin/
    rm terraform_1.13.0_linux_amd64.zip
else
    echo "Terraform is already installed."
fi
terraform --version

Terraform is already installed.
Terraform v1.13.0
on linux_amd64
+ provider registry.terraform.io/hashicorp/aws v5.100.0


In [2]:
%%writefile variables.tf
variable "region" {
  default = "us-east-1"
}

variable "vpc_cidr" {
  default = "10.0.0.0/16"
}

variable "public_subnet_cidr" {
  default = "10.0.0.0/24"
}

variable "private_subnet_1_cidr" {
  default = "10.0.1.0/24"
}

variable "private_subnet_2_cidr" {
  default = "10.0.2.0/24"
}

variable "az1" {
  default = "us-east-1a"
}

variable "az2" {
  default = "us-east-1b"
}

variable "mongodb_port" {
  default = 3307
}

Writing variables.tf


In [3]:
%%writefile provider.tf
terraform {
  required_providers {
    aws = {
      source  = "hashicorp/aws"
      version = "~> 5.0"
    }
  }
}

provider "aws" {
  region = var.region
}

Writing provider.tf


In [4]:
%%writefile outputs.tf
output "vpc_id" {
  value = aws_vpc.my_vpc.id
}

Writing outputs.tf


In [5]:
%%writefile vpc.tf
# VPC
resource "aws_vpc" "my_vpc" {
  cidr_block = var.vpc_cidr
  tags = {
    Name = "my-vpc"
  }
}

# Public Subnet
resource "aws_subnet" "public_subnet" {
  vpc_id            = aws_vpc.my_vpc.id
  cidr_block        = var.public_subnet_cidr
  availability_zone = var.az1
  tags = {
    Name = "public-subnet"
  }
}

# Private Subnet 1
resource "aws_subnet" "private_subnet_1" {
  vpc_id            = aws_vpc.my_vpc.id
  cidr_block        = var.private_subnet_1_cidr
  availability_zone = var.az1
  tags = {
    Name = "private-subnet-1"
  }
}

# Private Subnet 2
resource "aws_subnet" "private_subnet_2" {
  vpc_id            = aws_vpc.my_vpc.id
  cidr_block        = var.private_subnet_2_cidr
  availability_zone = var.az2
  tags = {
    Name = "private-subnet-2"
  }
}

# Internet Gateway
resource "aws_internet_gateway" "my_igw" {
  vpc_id = aws_vpc.my_vpc.id
  tags = {
    Name = "my-igw"
  }
}

# Elastic IP for NAT
resource "aws_eip" "nat_eip" {
  domain = "vpc"
}

# NAT Gateway in Public Subnet
resource "aws_nat_gateway" "my_nat" {
  allocation_id = aws_eip.nat_eip.id
  subnet_id     = aws_subnet.public_subnet.id
  tags = {
    Name = "my-nat"
  }
  depends_on = [aws_internet_gateway.my_igw]
}

# Public Route Table
resource "aws_route_table" "public_rt" {
  vpc_id = aws_vpc.my_vpc.id
  tags = {
    Name = "public-rt"
  }
}

resource "aws_route" "public_internet" {
  route_table_id         = aws_route_table.public_rt.id
  destination_cidr_block = "0.0.0.0/0"
  gateway_id             = aws_internet_gateway.my_igw.id
}

resource "aws_route_table_association" "public_assoc" {
  subnet_id      = aws_subnet.public_subnet.id
  route_table_id = aws_route_table.public_rt.id
}

# Private Route Table
resource "aws_route_table" "private_rt" {
  vpc_id = aws_vpc.my_vpc.id
  tags = {
    Name = "private-rt"
  }
}

resource "aws_route" "private_nat" {
  route_table_id         = aws_route_table.private_rt.id
  destination_cidr_block = "0.0.0.0/0"
  nat_gateway_id         = aws_nat_gateway.my_nat.id
}

resource "aws_route_table_association" "private_assoc_1" {
  subnet_id      = aws_subnet.private_subnet_1.id
  route_table_id = aws_route_table.private_rt.id
}

resource "aws_route_table_association" "private_assoc_2" {
  subnet_id      = aws_subnet.private_subnet_2.id
  route_table_id = aws_route_table.private_rt.id
}

Writing vpc.tf


In [6]:
%%bash
terraform init

Initializing the backend...
Initializing provider plugins...
- Reusing previous version of hashicorp/aws from the dependency lock file
- Using previously-installed hashicorp/aws v5.100.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [7]:
%%bash
terraform plan


Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_eip.nat_eip will be created
  + resource "aws_eip" "nat_eip" {
      + allocation_id        = (known after apply)
      + arn                  = (known after apply)
      + association_id       = (known after apply)
      + carrier_ip           = (known after apply)
      + customer_owned_ip    = (known after apply)
      + domain               = "vpc"
      + id                   = (known after apply)
      + instance             = (known after apply)
      + ipam_pool_id         = (known after apply)
      + network_border_group = (known after apply)
      + network_interface    = (known after apply)
      + private_dns          = (known after apply)
      + private_ip           = (known after apply)
      + ptr_record           = (known after apply)
      + public_dns           =

In [8]:
%%bash
terraform apply -auto-approve


Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_eip.nat_eip will be created
  + resource "aws_eip" "nat_eip" {
      + allocation_id        = (known after apply)
      + arn                  = (known after apply)
      + association_id       = (known after apply)
      + carrier_ip           = (known after apply)
      + customer_owned_ip    = (known after apply)
      + domain               = "vpc"
      + id                   = (known after apply)
      + instance             = (known after apply)
      + ipam_pool_id         = (known after apply)
      + network_border_group = (known after apply)
      + network_interface    = (known after apply)
      + private_dns          = (known after apply)
      + private_ip           = (known after apply)
      + ptr_record           = (known after apply)
      + public_dns           =

In [9]:
%%bash
# Get VPC ID from output
VPC_ID=$(terraform output -raw vpc_id)

# Verify VPC
aws ec2 describe-vpcs --vpc-ids $VPC_ID

# Verify Subnets
aws ec2 describe-subnets --filters "Name=vpc-id,Values=$VPC_ID"

# Verify Route Tables
aws ec2 describe-route-tables --filters "Name=vpc-id,Values=$VPC_ID"

# Verify IGW
aws ec2 describe-internet-gateways --filters "Name=attachment.vpc-id,Values=$VPC_ID"

# Verify NAT (should be 'available')
aws ec2 describe-nat-gateways --filter "Name=vpc-id,Values=$VPC_ID"

{
    "Vpcs": [
        {
            "OwnerId": "609009159737",
            "InstanceTenancy": "default",
            "CidrBlockAssociationSet": [
                {
                    "AssociationId": "vpc-cidr-assoc-01e8aa64001bccd0f",
                    "CidrBlock": "10.0.0.0/16",
                    "CidrBlockState": {
                        "State": "associated"
                    }
                }
            ],
            "IsDefault": false,
            "Tags": [
                {
                    "Key": "Name",
                    "Value": "my-vpc"
                }
            ],
            "BlockPublicAccessStates": {
                "InternetGatewayBlockMode": "off"
            },
            "VpcId": "vpc-09d06141d745ce35f",
            "State": "available",
            "CidrBlock": "10.0.0.0/16",
            "DhcpOptionsId": "dopt-0bf15e3dedd5548e4"
        }
    ]
}
{
    "Subnets": [
        {
            "AvailabilityZoneId": "use1-az6",
            "MapCusto

In [10]:
%%writefile nacl.tf
resource "aws_network_acl" "my_nacl" {
  vpc_id = aws_vpc.my_vpc.id
  tags = {
    Name = "my-nacl"
  }
}

# Inbound Allow All
resource "aws_network_acl_rule" "inbound_allow_all" {
  network_acl_id = aws_network_acl.my_nacl.id
  rule_number    = 100
  egress         = false
  protocol       = "-1"
  rule_action    = "allow"
  cidr_block     = "0.0.0.0/0"
}

# Outbound Allow All
resource "aws_network_acl_rule" "outbound_allow_all" {
  network_acl_id = aws_network_acl.my_nacl.id
  rule_number    = 100
  egress         = true
  protocol       = "-1"
  rule_action    = "allow"
  cidr_block     = "0.0.0.0/0"
}

# Associations
resource "aws_network_acl_association" "public_nacl_assoc" {
  subnet_id      = aws_subnet.public_subnet.id
  network_acl_id = aws_network_acl.my_nacl.id
}

resource "aws_network_acl_association" "private_1_nacl_assoc" {
  subnet_id      = aws_subnet.private_subnet_1.id
  network_acl_id = aws_network_acl.my_nacl.id
}

resource "aws_network_acl_association" "private_2_nacl_assoc" {
  subnet_id      = aws_subnet.private_subnet_2.id
  network_acl_id = aws_network_acl.my_nacl.id
}

Writing nacl.tf


In [11]:
!terraform plan

aws_vpc.my_vpc: Refreshing state... [id=vpc-09d06141d745ce35f]
aws_eip.nat_eip: Refreshing state... [id=eipalloc-01a7c8f7772f11809]
aws_subnet.private_subnet_2: Refreshing state... [id=subnet-0e0f1ec3d6fa55ba1]
aws_subnet.private_subnet_1: Refreshing state... [id=subnet-0a9ddef84ff975a4b]
aws_subnet.public_subnet: Refreshing state... [id=subnet-0122da2129d691098]
aws_internet_gateway.my_igw: Refreshing state... [id=igw-06334980ce938c2a0]
aws_route_table.public_rt: Refreshing state... [id=rtb-0c35f0d2e95f8b3ee]
aws_route_table.private_rt: Refreshing state... [id=rtb-0d592d6d93b2b0981]
aws_route.public_internet: Refreshing state... [id=r-rtb-0c35f0d2e95f8b3ee1080289494]
aws_route_table_association.private_assoc_2: Refreshing state... [id=rtbassoc-0369cf17e84ad5dd2]
aws_route_table_association.private_assoc_1: Refreshing state... [id=rtbassoc-09354b98ef22e6b5d]
aws_route_table_association.public_assoc: Refreshing state... [id=rtbassoc-01f7b02613d758fff]
aws_nat_gateway.my_nat: Refreshing 

In [12]:
!terraform apply --auto-approve

aws_eip.nat_eip: Refreshing state... [id=eipalloc-01a7c8f7772f11809]
aws_vpc.my_vpc: Refreshing state... [id=vpc-09d06141d745ce35f]
aws_internet_gateway.my_igw: Refreshing state... [id=igw-06334980ce938c2a0]
aws_route_table.private_rt: Refreshing state... [id=rtb-0d592d6d93b2b0981]
aws_subnet.public_subnet: Refreshing state... [id=subnet-0122da2129d691098]
aws_subnet.private_subnet_2: Refreshing state... [id=subnet-0e0f1ec3d6fa55ba1]
aws_route_table.public_rt: Refreshing state... [id=rtb-0c35f0d2e95f8b3ee]
aws_subnet.private_subnet_1: Refreshing state... [id=subnet-0a9ddef84ff975a4b]
aws_route_table_association.private_assoc_1: Refreshing state... [id=rtbassoc-09354b98ef22e6b5d]
aws_route.public_internet: Refreshing state... [id=r-rtb-0c35f0d2e95f8b3ee1080289494]
aws_route_table_association.private_assoc_2: Refreshing state... [id=rtbassoc-0369cf17e84ad5dd2]
aws_nat_gateway.my_nat: Refreshing state... [id=nat-07b8255225e45e0b0]
aws_route_table_association.public_assoc: Refreshing state

In [13]:
%%bash
VPC_ID=$(terraform output -raw vpc_id)

# Get NACL ID (filter by tag)
NACL_ID=$(aws ec2 describe-network-acls --filters "Name=vpc-id,Values=$VPC_ID" "Name=tag:Name,Values=my-nacl" --query "NetworkAcls[0].NetworkAclId" --output text)

# Verify NACL
aws ec2 describe-network-acls --network-acl-ids $NACL_ID

{
    "NetworkAcls": [
        {
            "Associations": [
                {
                    "NetworkAclAssociationId": "aclassoc-0387ab3f5d00dc778",
                    "NetworkAclId": "acl-044a1c4ecc41bc076",
                    "SubnetId": "subnet-0e0f1ec3d6fa55ba1"
                },
                {
                    "NetworkAclAssociationId": "aclassoc-04ea4662a4bd8450a",
                    "NetworkAclId": "acl-044a1c4ecc41bc076",
                    "SubnetId": "subnet-0a9ddef84ff975a4b"
                },
                {
                    "NetworkAclAssociationId": "aclassoc-033cf7be9b70f2539",
                    "NetworkAclId": "acl-044a1c4ecc41bc076",
                    "SubnetId": "subnet-0122da2129d691098"
                }
            ],
            "Entries": [
                {
                    "CidrBlock": "0.0.0.0/0",
                    "Egress": true,
                    "Protocol": "-1",
                    "RuleAction": "allow",
               

In [14]:
%%writefile security_groups.tf
# SageMaker Security Group
resource "aws_security_group" "sagemaker_sg" {
  name        = "sagemaker-sg"
  description = "For SageMaker"
  vpc_id      = aws_vpc.my_vpc.id

  egress {
    from_port   = 0
    to_port     = 0
    protocol    = "-1"
    cidr_blocks = ["0.0.0.0/0"]
  }

  tags = {
    Name = "sagemaker-sg"
  }
}

# EC2 Security Group
resource "aws_security_group" "ec2_sg" {
  name        = "ec2-sg"
  description = "For EC2 MongoDB"
  vpc_id      = aws_vpc.my_vpc.id

  ingress {
    from_port       = var.mongodb_port
    to_port         = var.mongodb_port
    protocol        = "tcp"
    security_groups = [aws_security_group.sagemaker_sg.id]
  }

  egress {
    from_port   = 0
    to_port     = 0
    protocol    = "-1"
    cidr_blocks = ["0.0.0.0/0"]
  }

  tags = {
    Name = "ec2-sg"
  }
}

Writing security_groups.tf


In [15]:
%%bash
terraform plan

aws_eip.nat_eip: Refreshing state... [id=eipalloc-01a7c8f7772f11809]
aws_vpc.my_vpc: Refreshing state... [id=vpc-09d06141d745ce35f]
aws_subnet.public_subnet: Refreshing state... [id=subnet-0122da2129d691098]
aws_subnet.private_subnet_1: Refreshing state... [id=subnet-0a9ddef84ff975a4b]
aws_route_table.private_rt: Refreshing state... [id=rtb-0d592d6d93b2b0981]
aws_subnet.private_subnet_2: Refreshing state... [id=subnet-0e0f1ec3d6fa55ba1]
aws_route_table.public_rt: Refreshing state... [id=rtb-0c35f0d2e95f8b3ee]
aws_internet_gateway.my_igw: Refreshing state... [id=igw-06334980ce938c2a0]
aws_network_acl.my_nacl: Refreshing state... [id=acl-044a1c4ecc41bc076]
aws_route.public_internet: Refreshing state... [id=r-rtb-0c35f0d2e95f8b3ee1080289494]
aws_network_acl_rule.outbound_allow_all: Refreshing state... [id=nacl-1970693416]
aws_network_acl_rule.inbound_allow_all: Refreshing state... [id=nacl-1847467266]
aws_network_acl_association.private_2_nacl_assoc: Refreshing state... [id=aclassoc-0387a

In [16]:
%%bash
terraform apply -auto-approve

aws_vpc.my_vpc: Refreshing state... [id=vpc-09d06141d745ce35f]
aws_eip.nat_eip: Refreshing state... [id=eipalloc-01a7c8f7772f11809]
aws_route_table.public_rt: Refreshing state... [id=rtb-0c35f0d2e95f8b3ee]
aws_route_table.private_rt: Refreshing state... [id=rtb-0d592d6d93b2b0981]
aws_network_acl.my_nacl: Refreshing state... [id=acl-044a1c4ecc41bc076]
aws_subnet.private_subnet_1: Refreshing state... [id=subnet-0a9ddef84ff975a4b]
aws_subnet.public_subnet: Refreshing state... [id=subnet-0122da2129d691098]
aws_subnet.private_subnet_2: Refreshing state... [id=subnet-0e0f1ec3d6fa55ba1]
aws_internet_gateway.my_igw: Refreshing state... [id=igw-06334980ce938c2a0]
aws_network_acl_rule.outbound_allow_all: Refreshing state... [id=nacl-1970693416]
aws_network_acl_rule.inbound_allow_all: Refreshing state... [id=nacl-1847467266]
aws_route.public_internet: Refreshing state... [id=r-rtb-0c35f0d2e95f8b3ee1080289494]
aws_route_table_association.private_assoc_2: Refreshing state... [id=rtbassoc-0369cf17e8

In [17]:
%%bash
VPC_ID=$(terraform output -raw vpc_id)

# Describe all SGs in VPC
aws ec2 describe-security-groups --filters "Name=vpc-id,Values=$VPC_ID"

{
    "SecurityGroups": [
        {
            "GroupId": "sg-08d3bc7e3f0057d80",
            "IpPermissionsEgress": [
                {
                    "IpProtocol": "-1",
                    "UserIdGroupPairs": [],
                    "IpRanges": [
                        {
                            "CidrIp": "0.0.0.0/0"
                        }
                    ],
                    "Ipv6Ranges": [],
                    "PrefixListIds": []
                }
            ],
            "Tags": [
                {
                    "Key": "Name",
                    "Value": "ec2-sg"
                }
            ],
            "VpcId": "vpc-09d06141d745ce35f",
            "SecurityGroupArn": "arn:aws:ec2:us-east-1:609009159737:security-group/sg-08d3bc7e3f0057d80",
            "OwnerId": "609009159737",
            "GroupName": "ec2-sg",
            "Description": "For EC2 MongoDB",
            "IpPermissions": [
                {
                    "IpProtocol": "tcp",


In [18]:
%%writefile iam.tf
# SageMaker Execution Role
resource "aws_iam_role" "sagemaker_role" {
  name = "SageMakerRole"
  assume_role_policy = jsonencode({
    Version = "2012-10-17"
    Statement = [
      {
        Action = "sts:AssumeRole"
        Effect = "Allow"
        Principal = {
          Service = "sagemaker.amazonaws.com"
        }
      },
    ]
  })
}

resource "aws_iam_role_policy_attachment" "sagemaker_full_access" {
  role       = aws_iam_role.sagemaker_role.name
  policy_arn = "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess"
}

# EC2 SSM Role
resource "aws_iam_role" "ec2_ssm_role" {
  name = "EC2SSMRole"
  assume_role_policy = jsonencode({
    Version = "2012-10-17"
    Statement = [
      {
        Action = "sts:AssumeRole"
        Effect = "Allow"
        Principal = {
          Service = "ec2.amazonaws.com"
        }
      },
    ]
  })
}

resource "aws_iam_role_policy_attachment" "ec2_ssm_access" {
  role       = aws_iam_role.ec2_ssm_role.name
  policy_arn = "arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore"
}

resource "aws_iam_instance_profile" "ec2_ssm_profile" {
  name = "EC2SSMProfile"
  role = aws_iam_role.ec2_ssm_role.name
}

Writing iam.tf


In [19]:
%%bash
terraform plan

aws_eip.nat_eip: Refreshing state... [id=eipalloc-01a7c8f7772f11809]
aws_vpc.my_vpc: Refreshing state... [id=vpc-09d06141d745ce35f]
aws_network_acl.my_nacl: Refreshing state... [id=acl-044a1c4ecc41bc076]
aws_subnet.public_subnet: Refreshing state... [id=subnet-0122da2129d691098]
aws_internet_gateway.my_igw: Refreshing state... [id=igw-06334980ce938c2a0]
aws_subnet.private_subnet_2: Refreshing state... [id=subnet-0e0f1ec3d6fa55ba1]
aws_subnet.private_subnet_1: Refreshing state... [id=subnet-0a9ddef84ff975a4b]
aws_route_table.public_rt: Refreshing state... [id=rtb-0c35f0d2e95f8b3ee]
aws_security_group.sagemaker_sg: Refreshing state... [id=sg-0485562c610ce365f]
aws_route_table.private_rt: Refreshing state... [id=rtb-0d592d6d93b2b0981]
aws_network_acl_rule.inbound_allow_all: Refreshing state... [id=nacl-1847467266]
aws_network_acl_rule.outbound_allow_all: Refreshing state... [id=nacl-1970693416]
aws_route.public_internet: Refreshing state... [id=r-rtb-0c35f0d2e95f8b3ee1080289494]
aws_secur

In [20]:
%%bash
terraform apply -auto-approve

aws_vpc.my_vpc: Refreshing state... [id=vpc-09d06141d745ce35f]
aws_eip.nat_eip: Refreshing state... [id=eipalloc-01a7c8f7772f11809]
aws_internet_gateway.my_igw: Refreshing state... [id=igw-06334980ce938c2a0]
aws_route_table.private_rt: Refreshing state... [id=rtb-0d592d6d93b2b0981]
aws_network_acl.my_nacl: Refreshing state... [id=acl-044a1c4ecc41bc076]
aws_route_table.public_rt: Refreshing state... [id=rtb-0c35f0d2e95f8b3ee]
aws_subnet.private_subnet_1: Refreshing state... [id=subnet-0a9ddef84ff975a4b]
aws_subnet.public_subnet: Refreshing state... [id=subnet-0122da2129d691098]
aws_subnet.private_subnet_2: Refreshing state... [id=subnet-0e0f1ec3d6fa55ba1]
aws_security_group.sagemaker_sg: Refreshing state... [id=sg-0485562c610ce365f]
aws_nat_gateway.my_nat: Refreshing state... [id=nat-07b8255225e45e0b0]
aws_network_acl_association.public_nacl_assoc: Refreshing state... [id=aclassoc-033cf7be9b70f2539]
aws_route.public_internet: Refreshing state... [id=r-rtb-0c35f0d2e95f8b3ee1080289494]
aw

In [21]:
%%bash
# Verify SageMaker Role
aws iam get-role --role-name SageMakerRole

# Verify EC2 SSM Role
aws iam get-role --role-name EC2SSMRole

# Verify Instance Profile
aws iam get-instance-profile --instance-profile-name EC2SSMProfile

{
    "Role": {
        "Path": "/",
        "RoleName": "SageMakerRole",
        "RoleId": "AROAY3S6GJY4TEOLB5OLW",
        "Arn": "arn:aws:iam::609009159737:role/SageMakerRole",
        "CreateDate": "2025-08-25T23:07:12Z",
        "AssumeRolePolicyDocument": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "sagemaker.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        },
        "MaxSessionDuration": 3600,
        "RoleLastUsed": {}
    }
}
{
    "Role": {
        "Path": "/",
        "RoleName": "EC2SSMRole",
        "RoleId": "AROAY3S6GJY4UXSERF2FW",
        "Arn": "arn:aws:iam::609009159737:role/EC2SSMRole",
        "CreateDate": "2025-08-25T23:07:12Z",
        "AssumeRolePolicyDocument": {
            "Version": "2012-10-17",
            "Statement": [
             

In [58]:
%%writefile sagemaker.tf
resource "aws_sagemaker_domain" "my_domain" {
  domain_name = "my-domain"
  auth_mode   = "IAM"
  vpc_id      = aws_vpc.my_vpc.id
  subnet_ids  = [aws_subnet.private_subnet_1.id, aws_subnet.private_subnet_2.id]

  default_user_settings {
    execution_role  = aws_iam_role.sagemaker_role.arn
    security_groups = [aws_security_group.sagemaker_sg.id]
  }
  default_space_settings {
    execution_role  = aws_iam_role.sagemaker_role.arn
    security_groups = [aws_security_group.sagemaker_sg.id]
  }
  app_network_access_type = "VpcOnly"
}

Overwriting sagemaker.tf


In [23]:
%%writefile -a outputs.tf

output "sagemaker_domain_id" {
  value = aws_sagemaker_domain.my_domain.id
}

Appending to outputs.tf


In [59]:
%%bash
terraform plan

data.aws_ami.amazon_linux: Reading...
aws_eip.nat_eip: Refreshing state... [id=eipalloc-01a7c8f7772f11809]
aws_iam_role.ec2_ssm_role: Refreshing state... [id=EC2SSMRole]
aws_vpc.my_vpc: Refreshing state... [id=vpc-09d06141d745ce35f]
aws_iam_role.sagemaker_role: Refreshing state... [id=SageMakerRole]
aws_iam_instance_profile.ec2_ssm_profile: Refreshing state... [id=EC2SSMProfile]
aws_iam_role_policy_attachment.ec2_ssm_access: Refreshing state... [id=EC2SSMRole-20250825230712598600000002]
aws_iam_role_policy_attachment.sagemaker_full_access: Refreshing state... [id=SageMakerRole-20250825230712552900000001]
aws_route_table.public_rt: Refreshing state... [id=rtb-0c35f0d2e95f8b3ee]
aws_subnet.private_subnet_1: Refreshing state... [id=subnet-0a9ddef84ff975a4b]
aws_route_table.private_rt: Refreshing state... [id=rtb-0d592d6d93b2b0981]
aws_network_acl.my_nacl: Refreshing state... [id=acl-044a1c4ecc41bc076]
aws_subnet.public_subnet: Refreshing state... [id=subnet-0122da2129d691098]
aws_internet

In [60]:
%%bash
terraform apply -auto-approve

data.aws_ami.amazon_linux: Reading...
aws_iam_role.ec2_ssm_role: Refreshing state... [id=EC2SSMRole]
aws_iam_role.sagemaker_role: Refreshing state... [id=SageMakerRole]
aws_eip.nat_eip: Refreshing state... [id=eipalloc-01a7c8f7772f11809]
aws_vpc.my_vpc: Refreshing state... [id=vpc-09d06141d745ce35f]
aws_iam_role_policy_attachment.sagemaker_full_access: Refreshing state... [id=SageMakerRole-20250825230712552900000001]
aws_iam_role_policy_attachment.ec2_ssm_access: Refreshing state... [id=EC2SSMRole-20250825230712598600000002]
aws_iam_instance_profile.ec2_ssm_profile: Refreshing state... [id=EC2SSMProfile]
data.aws_ami.amazon_linux: Read complete after 0s [id=ami-0309908417d7ea2d0]
aws_subnet.private_subnet_1: Refreshing state... [id=subnet-0a9ddef84ff975a4b]
aws_route_table.public_rt: Refreshing state... [id=rtb-0c35f0d2e95f8b3ee]
aws_subnet.private_subnet_2: Refreshing state... [id=subnet-0e0f1ec3d6fa55ba1]
aws_route_table.private_rt: Refreshing state... [id=rtb-0d592d6d93b2b0981]
aws_

In [26]:
%%writefile ec2.tf
data "aws_ami" "amazon_linux" {
  most_recent = true
  owners      = ["amazon"]

  filter {
    name   = "name"
    values = ["al2023-ami-*-x86_64"]
  }
}

resource "aws_instance" "my_ec2" {
  ami           = data.aws_ami.amazon_linux.id
  instance_type = "t2.micro"
  subnet_id     = aws_subnet.private_subnet_1.id
  vpc_security_group_ids = [aws_security_group.ec2_sg.id]
  iam_instance_profile   = aws_iam_instance_profile.ec2_ssm_profile.name
  associate_public_ip_address = false

  user_data = base64encode(<<-EOF
              #!/bin/bash
              yum update -y
              sudo tee /etc/yum.repos.d/mongodb-org-8.0.repo <<EOT
              [mongodb-org-8.0]
              name=MongoDB Repository
              baseurl=https://repo.mongodb.org/yum/amazon/2023/mongodb-org/8.0/x86_64/
              gpgcheck=1
              enabled=1
              gpgkey=https://pgp.mongodb.com/server-8.0.asc
              EOT
              sudo yum install -y mongodb-org
              sudo sed -i "s/port: 27017/port: ${var.mongodb_port}/" /etc/mongod.conf
              sudo sed -i "s/bindIp: 127.0.0.1/bindIp: 0.0.0.0/" /etc/mongod.conf
              sudo systemctl daemon-reload
              sudo systemctl start mongod
              sudo systemctl enable mongod
              EOF
  )

  tags = {
    Name = "my-ec2"
  }
}

Writing ec2.tf


In [27]:
%%writefile -a outputs.tf

output "ec2_private_ip" {
  value = aws_instance.my_ec2.private_ip
}

output "mongodb_port" {
  value = var.mongodb_port
}

Appending to outputs.tf


In [28]:
%%bash
terraform plan

aws_eip.nat_eip: Refreshing state... [id=eipalloc-01a7c8f7772f11809]
aws_iam_role.ec2_ssm_role: Refreshing state... [id=EC2SSMRole]
aws_vpc.my_vpc: Refreshing state... [id=vpc-09d06141d745ce35f]
aws_iam_role.sagemaker_role: Refreshing state... [id=SageMakerRole]
data.aws_ami.amazon_linux: Reading...
aws_iam_role_policy_attachment.ec2_ssm_access: Refreshing state... [id=EC2SSMRole-20250825230712598600000002]
aws_iam_instance_profile.ec2_ssm_profile: Refreshing state... [id=EC2SSMProfile]
aws_iam_role_policy_attachment.sagemaker_full_access: Refreshing state... [id=SageMakerRole-20250825230712552900000001]
aws_subnet.private_subnet_1: Refreshing state... [id=subnet-0a9ddef84ff975a4b]
aws_security_group.sagemaker_sg: Refreshing state... [id=sg-0485562c610ce365f]
aws_internet_gateway.my_igw: Refreshing state... [id=igw-06334980ce938c2a0]
data.aws_ami.amazon_linux: Read complete after 1s [id=ami-0309908417d7ea2d0]
aws_route_table.public_rt: Refreshing state... [id=rtb-0c35f0d2e95f8b3ee]
aws

In [29]:
%%bash
terraform apply -auto-approve

data.aws_ami.amazon_linux: Reading...
aws_iam_role.sagemaker_role: Refreshing state... [id=SageMakerRole]
aws_eip.nat_eip: Refreshing state... [id=eipalloc-01a7c8f7772f11809]
aws_vpc.my_vpc: Refreshing state... [id=vpc-09d06141d745ce35f]
aws_iam_role.ec2_ssm_role: Refreshing state... [id=EC2SSMRole]
aws_iam_role_policy_attachment.ec2_ssm_access: Refreshing state... [id=EC2SSMRole-20250825230712598600000002]
aws_iam_instance_profile.ec2_ssm_profile: Refreshing state... [id=EC2SSMProfile]
aws_iam_role_policy_attachment.sagemaker_full_access: Refreshing state... [id=SageMakerRole-20250825230712552900000001]
data.aws_ami.amazon_linux: Read complete after 0s [id=ami-0309908417d7ea2d0]
aws_route_table.private_rt: Refreshing state... [id=rtb-0d592d6d93b2b0981]
aws_route_table.public_rt: Refreshing state... [id=rtb-0c35f0d2e95f8b3ee]
aws_subnet.public_subnet: Refreshing state... [id=subnet-0122da2129d691098]
aws_internet_gateway.my_igw: Refreshing state... [id=igw-06334980ce938c2a0]
aws_networ

In [30]:
%%bash
# Get Instance ID (filter by tag)
INSTANCE_ID=$(aws ec2 describe-instances --filters "Name=tag:Name,Values=my-ec2" "Name=instance-state-name,Values=running" --query "Reservations[0].Instances[0].InstanceId" --output text)

# Describe Instance
aws ec2 describe-instances --instance-ids $INSTANCE_ID

# To check MongoDB (optional: use SSM to run command)
# First, ensure instance is registered in SSM: aws ssm describe-instance-information
# Then: aws ssm send-command --instance-ids $INSTANCE_ID --document-name "AWS-RunShellScript" --parameters '{"commands":["netstat -tuln | grep 3307"]}' 
# Wait and get output: aws ssm list-command-invocations --command-id <command-id> --details

{
    "Reservations": [
        {
            "ReservationId": "r-04f7fdecea1f65852",
            "OwnerId": "609009159737",
            "Groups": [],
            "Instances": [
                {
                    "Architecture": "x86_64",
                    "BlockDeviceMappings": [
                        {
                            "DeviceName": "/dev/xvda",
                            "Ebs": {
                                "AttachTime": "2025-08-25T23:15:49.000Z",
                                "DeleteOnTermination": true,
                                "Status": "attached",
                                "VolumeId": "vol-0297271ff40c2fb5c"
                            }
                        }
                    ],
                    "ClientToken": "terraform-20250825231547313000000001",
                    "EbsOptimized": false,
                    "EnaSupport": true,
                    "Hypervisor": "xen",
                    "IamInstanceProfile": {
                

In [31]:
%%bash
terraform output

ec2_private_ip = "10.0.1.12"
mongodb_port = 3307
sagemaker_domain_id = "d-jccm2rxiyrfp"
vpc_id = "vpc-09d06141d745ce35f"


In [32]:
%%writefile sagemaker_user.tf
resource "aws_sagemaker_user_profile" "ai_user" {
  domain_id         = aws_sagemaker_domain.my_domain.id
  user_profile_name = "ai-user"
  user_settings {
    execution_role = aws_iam_role.sagemaker_role.arn
    security_groups = [aws_security_group.sagemaker_sg.id]
  }
}

Writing sagemaker_user.tf


In [33]:
%%writefile -a outputs.tf

output "user_profile_name" {
  value = aws_sagemaker_user_profile.ai_user.user_profile_name
}

Appending to outputs.tf


In [34]:
%%bash
terraform plan

data.aws_ami.amazon_linux: Reading...
aws_eip.nat_eip: Refreshing state... [id=eipalloc-01a7c8f7772f11809]
aws_iam_role.sagemaker_role: Refreshing state... [id=SageMakerRole]
aws_vpc.my_vpc: Refreshing state... [id=vpc-09d06141d745ce35f]
aws_iam_role.ec2_ssm_role: Refreshing state... [id=EC2SSMRole]
aws_iam_role_policy_attachment.ec2_ssm_access: Refreshing state... [id=EC2SSMRole-20250825230712598600000002]
aws_iam_instance_profile.ec2_ssm_profile: Refreshing state... [id=EC2SSMProfile]
aws_iam_role_policy_attachment.sagemaker_full_access: Refreshing state... [id=SageMakerRole-20250825230712552900000001]
data.aws_ami.amazon_linux: Read complete after 1s [id=ami-0309908417d7ea2d0]
aws_route_table.public_rt: Refreshing state... [id=rtb-0c35f0d2e95f8b3ee]
aws_network_acl.my_nacl: Refreshing state... [id=acl-044a1c4ecc41bc076]
aws_subnet.private_subnet_2: Refreshing state... [id=subnet-0e0f1ec3d6fa55ba1]
aws_subnet.public_subnet: Refreshing state... [id=subnet-0122da2129d691098]
aws_intern

In [35]:
%%bash
terraform apply -auto-approve

data.aws_ami.amazon_linux: Reading...
aws_iam_role.sagemaker_role: Refreshing state... [id=SageMakerRole]
aws_vpc.my_vpc: Refreshing state... [id=vpc-09d06141d745ce35f]
aws_eip.nat_eip: Refreshing state... [id=eipalloc-01a7c8f7772f11809]
aws_iam_role.ec2_ssm_role: Refreshing state... [id=EC2SSMRole]
aws_iam_role_policy_attachment.sagemaker_full_access: Refreshing state... [id=SageMakerRole-20250825230712552900000001]
aws_iam_role_policy_attachment.ec2_ssm_access: Refreshing state... [id=EC2SSMRole-20250825230712598600000002]
aws_iam_instance_profile.ec2_ssm_profile: Refreshing state... [id=EC2SSMProfile]
aws_subnet.private_subnet_1: Refreshing state... [id=subnet-0a9ddef84ff975a4b]
aws_subnet.private_subnet_2: Refreshing state... [id=subnet-0e0f1ec3d6fa55ba1]
aws_internet_gateway.my_igw: Refreshing state... [id=igw-06334980ce938c2a0]
aws_network_acl.my_nacl: Refreshing state... [id=acl-044a1c4ecc41bc076]
aws_route_table.public_rt: Refreshing state... [id=rtb-0c35f0d2e95f8b3ee]
aws_subn

In [61]:
%%writefile sagemaker_spaces.tf
# Private Space
resource "aws_sagemaker_space" "private_space" {
  domain_id  = aws_sagemaker_domain.my_domain.id
  space_name = "private-notebook"

  ownership_settings {
    owner_user_profile_name = aws_sagemaker_user_profile.ai_user.user_profile_name
  }

  space_settings {
    app_type = "JupyterLab"

    jupyter_lab_app_settings {
      default_resource_spec {
        instance_type = "ml.t3.medium"
      }
    }
  }

  space_sharing_settings {
    sharing_type = "Private"
  }
}

# Shared Space
resource "aws_sagemaker_space" "shared_space" {
  domain_id  = aws_sagemaker_domain.my_domain.id
  space_name = "shared-notebook"

  ownership_settings {
    owner_user_profile_name = aws_sagemaker_user_profile.ai_user.user_profile_name
  }

  space_settings {
    app_type = "JupyterLab"

    jupyter_lab_app_settings {
      default_resource_spec {
        instance_type = "ml.t3.medium"
      }
    }
  }

  space_sharing_settings {
    sharing_type = "Shared"
  }
}

Overwriting sagemaker_spaces.tf


In [55]:
%%writefile -a outputs.tf

output "private_space_name" {
  value = aws_sagemaker_space.private_space.space_name
}

output "shared_space_name" {
  value = aws_sagemaker_space.shared_space.space_name
}

Appending to outputs.tf


In [62]:
%%bash
terraform plan

aws_eip.nat_eip: Refreshing state... [id=eipalloc-01a7c8f7772f11809]
aws_vpc.my_vpc: Refreshing state... [id=vpc-09d06141d745ce35f]
data.aws_ami.amazon_linux: Reading...
aws_iam_role.sagemaker_role: Refreshing state... [id=SageMakerRole]
aws_iam_role.ec2_ssm_role: Refreshing state... [id=EC2SSMRole]
aws_iam_role_policy_attachment.ec2_ssm_access: Refreshing state... [id=EC2SSMRole-20250825230712598600000002]
aws_iam_instance_profile.ec2_ssm_profile: Refreshing state... [id=EC2SSMProfile]
aws_iam_role_policy_attachment.sagemaker_full_access: Refreshing state... [id=SageMakerRole-20250825230712552900000001]
aws_subnet.private_subnet_1: Refreshing state... [id=subnet-0a9ddef84ff975a4b]
aws_subnet.public_subnet: Refreshing state... [id=subnet-0122da2129d691098]
aws_route_table.public_rt: Refreshing state... [id=rtb-0c35f0d2e95f8b3ee]
aws_security_group.sagemaker_sg: Refreshing state... [id=sg-0485562c610ce365f]
aws_internet_gateway.my_igw: Refreshing state... [id=igw-06334980ce938c2a0]
aws_

In [63]:
%%bash
terraform apply -auto-approve

data.aws_ami.amazon_linux: Reading...
aws_eip.nat_eip: Refreshing state... [id=eipalloc-01a7c8f7772f11809]
aws_iam_role.sagemaker_role: Refreshing state... [id=SageMakerRole]
aws_iam_role.ec2_ssm_role: Refreshing state... [id=EC2SSMRole]
aws_vpc.my_vpc: Refreshing state... [id=vpc-09d06141d745ce35f]
aws_iam_role_policy_attachment.sagemaker_full_access: Refreshing state... [id=SageMakerRole-20250825230712552900000001]
aws_iam_role_policy_attachment.ec2_ssm_access: Refreshing state... [id=EC2SSMRole-20250825230712598600000002]
aws_iam_instance_profile.ec2_ssm_profile: Refreshing state... [id=EC2SSMProfile]
data.aws_ami.amazon_linux: Read complete after 0s [id=ami-0309908417d7ea2d0]
aws_subnet.private_subnet_2: Refreshing state... [id=subnet-0e0f1ec3d6fa55ba1]
aws_route_table.public_rt: Refreshing state... [id=rtb-0c35f0d2e95f8b3ee]
aws_network_acl.my_nacl: Refreshing state... [id=acl-044a1c4ecc41bc076]
aws_security_group.sagemaker_sg: Refreshing state... [id=sg-0485562c610ce365f]
aws_rou